In [3]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
import json
load_dotenv(find_dotenv())

True

In [4]:
llm = ChatOpenAI(
    model="gpt-5-nano",
    # stream_usage=True,
    # temperature=None,
    # max_tokens=None,
    # timeout=None,
    # reasoning_effort="low",
    # max_retries=2,
    # api_key="...",  # If you prefer to pass api key in directly
    # base_url="...",
    # organization="...",
    # other params...
)

In [ ]:
generate_system_message = '''
You are FAANGFocus, an AI that generates adaptive coding interview questions.

OUTPUT REQUIREMENTS:
- Output ONLY a valid JSON object
- No markdown, no commentary, no text before or after
- Strict adherence to the schema below

COMPANY-SPECIFIC THEMING (CRITICAL):
Every element of the problem MUST reflect the specified company's domain:
- Pinterest: boards, pins, users, follows, saves, recommendations
- Meta: friends, posts, social graph, comments, reactions, feeds
- Amazon: products, warehouses, shipments, orders, inventory
- Google: search queries, documents, rankings, indexing
- Netflix: movies, watch history, recommendations, ratings
- Uber: rides, drivers, locations, routes, pricing
The problem must feel authentic to that company's interview style.

MANDATORY JSON SCHEMA:

{
  "company": string,
  "title": string,
  "description": string,
  "inputs": [
    { "name": string, "type": string, "description": string }
  ],
  "examples": [
    { "input": object, "output": any, "explanation": string }
  ],
  "constraints": [ string ],
  "goal": string,
  "difficulty": "easy" | "medium" | "hard",
  "function_name": string,
  "function_signature": string,
  "hints": [ string ],
  "test_cases": [
    { "input": object, "output": any }
  ]
}

CRITICAL RULES:

1. NEWLINES:
   - Use actual newline characters in all text fields
   - NEVER output literal "\n" strings
   - Multi-line text fields: description, explanation, constraints, goal, hints

2. INPUT CONSISTENCY:
   - Variable names in "inputs" must match exactly across:
     * "function_signature" parameters
     * "examples[].input" keys
     * "test_cases[].input" keys
   - Type annotations in "inputs[].type" must match "function_signature"
   
   Example:
   inputs: [{ "name": "boards", "type": "List[List[int]]" }]
   function_signature: "def cluster_boards(boards: List[List[int]]):"
   examples/test_cases: { "boards": [[...]] }

3. FUNCTION SIGNATURE:
   - Valid Python syntax only
   - Parameters in same order as "inputs"
   - No function body, only the "def ...:" line
   - Must include type hints matching "inputs"

4. FUNCTION NAMING:
   - MUST be semantically meaningful (NOT "solve")
   - Use snake_case
   - Reflect the actual operation (e.g., find_common_friends, rank_search_results, optimize_delivery_routes)
   - Must match between "function_name" and "function_signature"

5. EXAMPLES:
   - EXACTLY 2 examples required
   - Example 1: Normal/typical case
   - Example 2: Edge case (empty input, minimal input, boundary condition)

6. HINTS:
   - Maximum 5 hints allowed
   - Write hints as if an interviewer is speaking directly to the candidate
   - Use second person ("you", "your") and conversational language
   - Sound natural, encouraging, and guiding (not robotic or formal)
   - Progressive difficulty: start gentle, get more specific

7. TEST CASES:
   - Between 7 and 12 test cases
   - Cover: normal cases, edge cases, boundary conditions, large inputs

8. JSON VALIDITY:
   - No trailing commas
   - Proper escaping of quotes and special characters
   - All strings properly quoted
   - No additional fields beyond schema

SCHEMA COMPLIANCE:
- Do not add, remove, or rename any fields
- Field order and structure must match exactly
- All fields are required (none are optional)

Generate your response now.
'''

generate_human_message = "I am interviewing for uber swe intern and someone before me got a graph problem that was easy to medium difficulty"

In [21]:
messages = [
    ("system", system_message), ("human", human_message)
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='{\n  "company": "Uber",\n  "title": "Limited-stop cheapest ride routing on a city graph",\n  "description": "You are building a feature to suggest the cheapest ride path from a rider\'s pickup to their dropoff in a city road graph. Each road segment is a directed edge with a distance (in kilometers) and an optional toll. The fare for traversing an edge is distance * per_meter_rate plus toll (default toll is 0). The rider can be routed through at most max_stops intermediate locations (i.e., transfers). Return the minimum total fare to reach the destination using at most max_stops transfers. If no such route exists, return -1. The problem models a common Uber routing scenario where you balance shortest fare with a limit on the number of detours or transfers.",\n  "inputs": [\n    { "name": "edges", "type": "List[Dict[str, Any]]", "description": "A list of directed edges representing the city road graph. Each edge has fields: from (str), to (str), distance (float) in ki

In [23]:
problem_json = ai_msg.text
# output = problem_json.replace("\\n","")
output = problem_json.replace("\n","")
output = json.loads(output)
output

{'company': 'Uber',
 'title': 'Limited-stop cheapest ride routing on a city graph',
 'description': "You are building a feature to suggest the cheapest ride path from a rider's pickup to their dropoff in a city road graph. Each road segment is a directed edge with a distance (in kilometers) and an optional toll. The fare for traversing an edge is distance * per_meter_rate plus toll (default toll is 0). The rider can be routed through at most max_stops intermediate locations (i.e., transfers). Return the minimum total fare to reach the destination using at most max_stops transfers. If no such route exists, return -1. The problem models a common Uber routing scenario where you balance shortest fare with a limit on the number of detours or transfers.",
 'inputs': [{'name': 'edges',
   'type': 'List[Dict[str, Any]]',
   'description': 'A list of directed edges representing the city road graph. Each edge has fields: from (str), to (str), distance (float) in kilometers, toll (float, optional

In [24]:
output['company']

'Uber'

In [26]:
output['title']

'Limited-stop cheapest ride routing on a city graph'

In [27]:
output['description']

"You are building a feature to suggest the cheapest ride path from a rider's pickup to their dropoff in a city road graph. Each road segment is a directed edge with a distance (in kilometers) and an optional toll. The fare for traversing an edge is distance * per_meter_rate plus toll (default toll is 0). The rider can be routed through at most max_stops intermediate locations (i.e., transfers). Return the minimum total fare to reach the destination using at most max_stops transfers. If no such route exists, return -1. The problem models a common Uber routing scenario where you balance shortest fare with a limit on the number of detours or transfers."

In [28]:
output['inputs']

[{'name': 'edges',
  'type': 'List[Dict[str, Any]]',
  'description': 'A list of directed edges representing the city road graph. Each edge has fields: from (str), to (str), distance (float) in kilometers, toll (float, optional, default 0).'},
 {'name': 'start',
  'type': 'str',
  'description': 'Starting location identifier.'},
 {'name': 'end',
  'type': 'str',
  'description': 'Destination location identifier.'},
 {'name': 'per_meter_rate',
  'type': 'float',
  'description': 'Cost per meter traveled, in the same currency as toll.'},
 {'name': 'max_stops',
  'type': 'int',
  'description': 'Maximum number of intermediate stops allowed (i.e., transfers).'}]

In [29]:
output['examples']

[{'input': {'edges': [{'from': 'A', 'to': 'B', 'distance': 5, 'toll': 1},
    {'from': 'B', 'to': 'C', 'distance': 10},
    {'from': 'A', 'to': 'C', 'distance': 15, 'toll': 0}],
   'start': 'A',
   'end': 'C',
   'per_meter_rate': 2.0,
   'max_stops': 1},
  'output': 30.0,
  'explanation': 'Direct edge A->C costs 15*2 = 30. The two-hop path A->B->C costs  (5*2+1) + (10*2+0) = 31, which is larger. With at most 1 stop, the direct path is the cheapest.'},
 {'input': {'edges': [{'from': 'A', 'to': 'B', 'distance': 3},
    {'from': 'B', 'to': 'C', 'distance': 4}],
   'start': 'A',
   'end': 'C',
   'per_meter_rate': 1.0,
   'max_stops': 0},
  'output': -1,
  'explanation': 'No direct edge A->C exists, and max_stops is 0, so no valid route is possible.'}]

In [30]:
output['constraints']

['Edges are non-negative distances; tolls are non-negative.',
 'Graph may contain cycles; ensure algorithm terminates and uses at most max_stops stops.',
 'If multiple paths have the same cost, returning any one valid path with that cost is acceptable.',
 'If no path satisfies the max_stops constraint, return -1.']

In [31]:
output['goal']

'Return the minimum total fare to travel from start to end using at most max_stops intermediate locations.'

In [32]:
output['function_name']

'find_cheapest_route_within_stops'

In [33]:
output['function_signature']

'def find_cheapest_route_within_stops(edges: List[Dict[str, Any]], start: str, end: str, per_meter_rate: float, max_stops: int):'

In [34]:
output['hints']

['Think in terms of shortest path with a constraint on the number of stops; augment state with the number of stops used.',
 'Use a priority queue (Dijkstra-like) where each state is (cost, node, stops_so_far).',
 'Edge cost is edge.distance * per_meter_rate + edge.toll (default toll 0).',
 'Maintain a best_costs dictionary or matrix to prune dominated states (lower cost for same node with <= stops).',
 'If you exhaust all states without reaching end within max_stops, return -1.']

In [35]:
solve_system_message = '''
You are an expert software engineer solving coding interview problems.

INPUT:
You will receive a JSON object containing a coding problem with:
- Problem description
- Function signature
- Input/output examples
- Test cases
- Hints
- Constraints

YOUR TASK:
Write a complete, working solution to the problem.

OUTPUT REQUIREMENTS:
- Return ONLY the Python function code
- No markdown formatting, no code block markers (no ```python```)
- No explanatory text before or after the code
- Just the raw Python function, ready to execute

CODE QUALITY STANDARDS:

1. COMMENTS:
   - Add a docstring explaining what the function does
   - Include inline comments explaining key logic steps
   - Comment on non-obvious optimizations or algorithm choices
   - Explain complex operations or data structure usage
   - Keep comments concise but meaningful

2. IMPLEMENTATION:
   - Use the exact function signature provided in the JSON
   - Follow Python best practices and conventions
   - Prioritize clarity and readability
   - Use descriptive variable names
   - Aim for optimal time/space complexity where reasonable

3. STYLE:
   - Clean, well-structured code
   - Proper indentation and spacing
   - Handle edge cases mentioned in constraints
   - No unnecessary complexity

COMMENT STYLE EXAMPLES:

Good:
"""
Finds the intersection of two sorted arrays.
Returns elements that appear in both arrays.
Time: O(n + m), Space: O(min(n, m))
"""
def find_intersection(arr1: List[int], arr2: List[int]) -> List[int]:
    # Use two pointers to traverse both arrays simultaneously
    i, j = 0, 0
    result = []
    
    while i < len(arr1) and j < len(arr2):
        if arr1[i] == arr2[j]:
            # Found common element
            result.append(arr1[i])
            i += 1
            j += 1
        elif arr1[i] < arr2[j]:
            i += 1
        else:
            j += 1
    
    return result

Bad (too verbose):
# This function finds intersection
# It takes two arrays
# It returns common elements
def find_intersection(arr1: List[int], arr2: List[int]) -> List[int]:
    # Initialize i to 0
    i = 0
    # Initialize j to 0
    j = 0
    # Create empty result list
    result = []
    # Loop while both pointers are valid
    while i < len(arr1) and j < len(arr2):
        # Check if elements are equal
        if arr1[i] == arr2[j]:
            # If equal, append to result
            result.append(arr1[i])
            # Increment i
            i += 1
            # Increment j
            j += 1
    # Return the result
    return result

APPROACH:
1. Read and understand the problem from the JSON
2. Consider the hints provided
3. Implement an efficient solution
4. Add clear, helpful comments
5. Ensure the code handles all test cases and edge cases

Generate the solution now.
'''

solve_human_message = problem_json

In [37]:
solve_messages = [
    ("system", solve_system_message), ("human", solve_human_message)
]
solution = llm.invoke(solve_messages)
solution.text

'def find_cheapest_route_within_stops(edges: List[Dict[str, Any]], start: str, end: str, per_meter_rate: float, max_stops: int):\n    """\n    Find the minimum fare to travel from start to end with at most max_stops intermediate locations.\n    Each edge cost is: distance * per_meter_rate + toll (default toll = 0).\n    If no valid route exists within the stops constraint, return -1.\n    """\n    from typing import List, Dict, Any\n    import heapq\n\n    # Build adjacency list: from_node -> list of (to, distance, toll)\n    graph: Dict[str, List[tuple]] = {}\n    for e in edges:\n        u = e.get("from")\n        v = e.get("to")\n        dist = e.get("distance", 0.0)\n        toll = e.get("toll", 0.0)\n        graph.setdefault(u, []).append((v, dist, toll))\n\n    # If start equals end, zero-cost path is valid (no travel)\n    if start == end:\n        return 0.0\n\n    # Priority queue for Dijkstra-like search: (cost, node, edges_used)\n    # edges_used = number of edges traversed 